In [1]:
import sys
# the mock-0.3.1 dir contains testcase.py, testutils.py & mock.py
sys.path.append('/Users/htian/Documents/projects/ploth/')
import ploth as pth
# from testcase import TestCase
# from testutils import RunTests
# from mock import Mock, sentinel, patch

In [2]:
%matplotlib inline
# common functions used in the project of VR-R in GAC direction
import numpy as np
#import scipy.linalg as splin
import numpy.linalg as nl
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('xtick',labelsize=20)
matplotlib.rc('ytick',labelsize=20)
font = {'size' : 20}
matplotlib.rc('font', **font)
from matplotlib import cm
from matplotlib.colors import Normalize
from scipy.optimize import curve_fit
import scipy.stats as stats
import scipy.special as special
import astropy.io.fits as fits
import emcee
import corner
import time
import galpy.util.bovy_coords as gub
# f = open('fevrvtvz.dat')
# VT = f.readlines()
# f.close()

In [3]:
def lnprob_gauss2(x,y):
    #n = np.float(len(y))
    f1 = x[0]
    mu1 = x[1]
    sig1 = x[2]
    mu2 = x[3]
    sig2 = x[4]
    if np.isinf(f1) or np.isinf(mu1) or np.isinf(sig1) or\
                np.isinf(mu2) or np.isinf(sig2) or\
                f1<0 or f1>1 or sig1<0 or sig2<0 or\
                sig1>200 or sig2>200 or\
                mu1>100 or mu1<-50 or mu2>300 or mu2<100:
        return -1e100
    g1 = f1*stats.norm.pdf(y,mu1,sig1)# np.exp(-(y-mu1)**2/(2*sig1**2))/(np.sqrt(2*np.pi)*sig1)
    g2 = (1-f1)*stats.norm.pdf(y,mu2,sig2)#np.exp(-(y-mu2)**2/(2*sig2**2))/(np.sqrt(2*np.pi)*sig2)
    g = g1+g2
    #print g
    ind_g = (np.isinf(g)==False) & (np.isnan(g)==False) & (g>0)
    logg = np.log(g[ind_g])
    ind_lg = logg>-1e100
    return np.sum(logg[ind_lg])

def gauss2_model(y,p0,N, func, nwalkers):
    # MCMC sampling
    ndim = p0.shape[1]
#     print(ndim)
    sampler = emcee.EnsembleSampler(nwalkers, \
            ndim, func, args=[y])
    N_burn_in = 2000
    
    pos, prob, state = sampler.run_mcmc(p0, N_burn_in)
    sampler.reset()
    
    sampler.run_mcmc(pos, N)
    
    samples = sampler.chain[:, N_burn_in:, :].reshape((-1, ndim))
    #corner.corner(samples)
    popt = np.median(samples, axis=0)
    pcov = np.zeros((ndim,ndim))
    for i in range(ndim):
        for j in range(ndim):
            pcov[i,j] = (np.sum((samples[:,i]-popt[i])*\
                (samples[:,j]-popt[j])))/len(samples)
    return popt, pcov, samples

def lnprob_gauss3(x,y):
    #n = np.float(len(y))
    f1 = x[0]
    f2 = x[1]
    mu1 = x[2]
    sig1 = x[3]
    mu2 = x[4]
    sig2 = x[5]
    mu3 = x[6]
    sig3 = x[7]
    
    if np.isinf(f1) or np.isinf(f2) or np.isinf(mu1) or np.isinf(sig1) or\
                np.isinf(mu2) or np.isinf(sig2) or np.isinf(mu3) or np.isinf(sig3) or\
                f1<0 or f1>1 or f2<0 or f2>1 or (f1+f2)<0 or (f1+f2)>1 or\
                sig1<0 or sig2<0 or sig3<0 or\
                sig1>200 or sig2>200 or sig3>200 or\
                mu1>100 or mu1<-100 or mu3>0 or mu3<-200 or mu2>300 or mu2<100:
        return -1e100
    g1 = f1*stats.norm.pdf(y,mu1,sig1)# np.exp(-(y-mu1)**2/(2*sig1**2))/(np.sqrt(2*np.pi)*sig1)
    g2 = f2*stats.norm.pdf(y,mu2,sig2)#np.exp(-(y-mu2)**2/(2*sig2**2))/(np.sqrt(2*np.pi)*sig2)
    g3 = (1-f1-f2)*stats.norm.pdf(y,mu3,sig3)#np.exp(-(y-mu2)**2/(2*sig2**2))/(np.sqrt(2*np.pi)*sig2)
    g = g1+g2+g3
    #print g
    ind_g = (np.isinf(g)==False) & (np.isnan(g)==False) & (g>0)
    logg = np.log(g[ind_g])
    ind_lg = logg>-1e100
    return np.sum(logg[ind_lg])

In [4]:
dpath = "/Users/htian/Documents/work/data/rothalo/"
ppath = "/Users/htian/Documents/work/plots/rothalo/"
fn = "dr5GaiaDR2_v1_all_photos_EWs_dist_halo_feh_le_m1_TRUEKGIANT_NODUPLICATED"
# fn = "dr5GaiaDR2_v1_all_photos_EWs_dist_halo_feh_le_m1_TRUEKGIANT_NODUPLICATED"
d_fmt = ".fits"
U_sun, V_sun, W_sun = 11.1, 12.24, 7.25
X_sun = 8.3
V_LSR = 232#232#218#232
fn_tag = "r2"#"vlsr218_"#"dGaia_" #"vlsr218_"  240 Reid et al 2014 

In [5]:
DtG = fits.open(dpath+fn+d_fmt)
dataG = DtG[1].data

ra_Ga, dec_Ga = dataG["ra_epoch2000_1"],dataG["dec_epoch2000_1"]
br_rpa, MagGa, a_ga = dataG["bp_rp_1"],dataG["phot_g_mean_mag_1"],dataG["a_g_val_1"]
rvGa, rvGea = dataG["radial_velocity_1"],dataG["radial_velocity_error_1"]
rvLa, rvLea = dataG["rv_1"],dataG["rv_err_1"]
feha = dataG["feh_1"]
pmra_Gae, pmdec_Gae = dataG["pmra_error_1"],dataG["pmdec_error_1"]
pmra_Ga, pmdec_Ga = dataG["pmra_1"],dataG["pmdec_1"]
feha,fehea = dataG["feh_1"],dataG["feh_err_1"]
dista, dista_15, dista_85,dista_len = dataG["r_est_1"],dataG["r_lo_1"],dataG["r_hi_1"],dataG["r_len_1"]
distae = 0.5*(dista_85 - dista_15)
prlxa, prlxae = dataG["parallax_1"],dataG["parallax_error_1"]
distLa,distLa_15,distLa_85 = dataG["DistK50_RJCE_1"],dataG["DistK15_RJCE_1"],dataG["DistK85_RJCE_1"]
snrga = dataG["snrg_1"]
# --------------------------------------------------------------------------------------------------------
ind_halo = (pmra_Ga ==pmra_Ga) & ( pmdec_Ga ==pmdec_Ga) &\
(rvGa==rvGa) & (rvGea==rvGea) & (dista==dista) #& (snrga>10)
ra_Gt,dec_Gt = ra_Ga[ind_halo],dec_Ga[ind_halo]
pmra_Gt,pmdec_Gt = pmra_Ga[ind_halo],pmdec_Ga[ind_halo]
pmra_eGt,pmdec_eGt = pmra_Gae[ind_halo],pmdec_Gae[ind_halo]
rvGt, rvGet = rvGa[ind_halo], rvGea[ind_halo]
rvLt, rvLet = rvLa[ind_halo], rvLea[ind_halo]
feht,fehet = feha[ind_halo],fehea[ind_halo]
distt, distet = dista[ind_halo]/1000, distae[ind_halo]/1000
# distt, distet = 1/prlxa[ind_halo], prlxae[ind_halo]/(prlxa[ind_halo]**2)
prlxt, prlxet = prlxa[ind_halo], prlxae[ind_halo]
distLt,distL_15t,distL_85t = distLa[ind_halo],distLa_15[ind_halo],distLa_85[ind_halo]
print(len(feht))
#--------------------------------------------------------
ind_halo0 =  (distet/distt<0.2) & (distt<4) 
ra_G,dec_G = ra_Gt[ind_halo0],dec_Gt[ind_halo0]
pmra_G,pmdec_G = pmra_Gt[ind_halo0],pmdec_Gt[ind_halo0]
pmra_eG,pmdec_eG = pmra_eGt[ind_halo0],pmdec_eGt[ind_halo0]
rvG, rvGe = rvGt[ind_halo0], rvGet[ind_halo0]
rvL, rvLe = rvLt[ind_halo0], rvLet[ind_halo0]
feh,fehe = feht[ind_halo0],fehet[ind_halo0]
dist, diste = distt[ind_halo0], distet[ind_halo0]
prlx, prlxe = prlxt[ind_halo0], prlxet[ind_halo0]
distL,distL_15,distL_85 = distLt[ind_halo0],distL_15t[ind_halo0],distL_85t[ind_halo0]
print(len(feh[(pmra_eG<0.1) & (pmdec_eG<0.1)]))
print(len(feh[rvGe<2]))
print(len(feh))

6660
3517
3054
3827


In [6]:
llbb = gub.radec_to_lb(ra_G,dec_G,degree=True)
# centered at LSR, with galactic center at (X_sun,0,0)kpc
xyz = gub.lbd_to_XYZ(llbb[:,0],llbb[:,1],dist,degree=True)  
pmllbb = gub.pmrapmdec_to_pmllpmbb(pmra_G,pmdec_G,ra_G,dec_G,degree=True)
vxvyvz = gub.vrpmllpmbb_to_vxvyvz(rvG,pmllbb[:,0],pmllbb[:,1],llbb[:,0],llbb[:,1],\
                                  dist,degree=True)
# vrvtvz0 = gub.vxvyvz_to_galcencyl(vxvyvz[:,0],vxvyvz[:,1],vxvyvz[:,2],xyz[:,0],\
#                                   X_sun-xyz[:,1],xyz[:,2],[0,220,0])
# here the sun's uvw in GC frame, XYZ in gc frame, UVW in LSR frame
vrvtvz = gub.vxvyvz_to_galcencyl(vxvyvz[:,0],vxvyvz[:,1],vxvyvz[:,2],\
                                 X_sun-xyz[:,0],xyz[:,1],xyz[:,2],[U_sun*(-1),V_LSR+V_sun,W_sun])  

In [7]:
nwalkers = 50
ndim = 8
p0=np.zeros((nwalkers,ndim))
p0[:,0] = np.random.rand(nwalkers)*0.2+0.5
p0[:,1] = np.random.rand(nwalkers)*0.2
p0[:,2] = np.random.rand(nwalkers)*100-50
p0[:,3] = np.random.rand(nwalkers)*40+60
p0[:,4] = np.random.rand(nwalkers)*60+140
p0[:,5] = np.random.rand(nwalkers)*50+20
p0[:,6] = np.random.rand(nwalkers)*60-80
p0[:,7] = np.random.rand(nwalkers)*200
start = time.time()


In [8]:
sn = 200    # times for MC 
rcd_mc = np.zeros((sn,8))
rcd_err = np.zeros((sn,8))
start = time.time()
for i in range(0,sn):
    if ((i % 20) == 0):
        print(i,time.time()-start) 
    s_pmra = np.random.normal(pmra_G,pmra_eG)
    s_pmdec = np.random.normal(pmdec_G,pmdec_eG)
    s_rv = np.random.normal(rvG,rvGe)
    s_dist = np.random.normal(dist,diste)
    
    s_xyz = gub.lbd_to_XYZ(llbb[:,0],llbb[:,1],s_dist,degree=True)  
    s_pmllbb = gub.pmrapmdec_to_pmllpmbb(s_pmra,s_pmdec,ra_G,dec_G,degree=True)
    s_vxvyvz = gub.vrpmllpmbb_to_vxvyvz(s_rv,s_pmllbb[:,0],s_pmllbb[:,1],llbb[:,0],llbb[:,1],\
                                      s_dist,degree=True)
    # here the sun's uvw in GC frame, XYZ in gc frame, UVW in LSR frame
    s_vrvtvz = gub.vxvyvz_to_galcencyl(s_vxvyvz[:,0],s_vxvyvz[:,1],s_vxvyvz[:,2],\
                                     X_sun-s_xyz[:,0],s_xyz[:,1],s_xyz[:,2],[U_sun*(-1),V_LSR+V_sun,W_sun])
#     plt.hist(s_vrvtvz[:,1],100)
#     plt.show()
    s_vt = s_vrvtvz[:,1]
    Mpopt3,Mpcov3, samples3 = gauss2_model(s_vt, p0, 5000, lnprob_gauss3, nwalkers)
    rcd_mc[i,:] = Mpopt3
    rcd_err[i,:] = np.sqrt(np.diag(Mpcov3))
print(time.time()-start)
np.savetxt(dpath+'rcd_mc4.dat',rcd_mc,delimiter=' ',fmt='%f')
np.savetxt(dpath+'rcd_err4.dat',rcd_err,delimiter=' ',fmt='%f')

0 0.00017523765563964844
20 3333.8798689842224
40 6640.069367170334
60 9959.561291217804
80 13311.772409200668
100 16635.650900125504
120 19962.74926519394
140 23298.654885292053
160 26602.442634105682
180 29921.55466222763
33234.703238248825


In [9]:
# sm = np.random.uniform(0,1,10)
# ss = np.random.uniform(0,1,10)*0+1
# nn = np.zeros_like(ss)+1000
# sv = np.random.normal(sm,ss)
# print(sm)
# print('++++++++++++++++++++++++')
# print(sv)

In [10]:
print(Mpopt3)

[  0.72345244   0.23000114  27.91220518  76.61435822 180.17307534
  46.67313984 -87.34092085 119.78493165]
